In [1]:
import requests, json
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import asyncio
import os
import json
import copy

from anchorpy import Wallet
from anchorpy import Provider
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient

from driftpy.constants.config import configs
from driftpy.types import *
#MarketType, OrderType, OrderParams, PositionDirection, OrderTriggerCondition

from driftpy.clearing_house import ClearingHouse
from driftpy.constants.numeric_constants import BASE_PRECISION, PRICE_PRECISION

from driftpy.constants.config import configs
from anchorpy import Provider
from anchorpy import Wallet
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.accounts import *
from solana.keypair import Keypair
from driftpy.math.positions import is_available
from driftpy.constants.numeric_constants import *
import os
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.clearing_house import ClearingHouse
from solana.keypair import Keypair
from driftpy.constants.config import configs
import os
from anchorpy import Wallet, Provider
from solana.rpc.async_api import AsyncClient

from simple_mm import *


MARKET_INDEX_SOL = 0
userAccount = "ECbtn9Y34m6L89EBDMRYxhpyrFgSHEcbELgozpqAvcDY"
PKEY = "7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t"
URL_ORDERS_SELF   = f"https://master.api.drift.trade/orders/userAccounts?userPublicKeys={PKEY}&pageIndex=0&pageSize=100&marketType=perp"
## (below), putting public key only allows me to see
URL_HISTORY_SELF  = f"https://beta.drift.trade/overview/history/tradeHistory?userAccount={PKEY}"
URL_DEVNET_OB     = "https://master.dlob.drift.trade/orders/json"

In [2]:
from driftpy import constants
constants.markets.devnet_markets

[Market(symbol='SOL-PERP', base_asset_symbol='SOL', market_index=0, pyth_oracle=J83w4HKfqxwcq3BEMMkPFSppX3gqekLyLJBexebFVkix),
 Market(symbol='BTC-PERP', base_asset_symbol='BTC', market_index=1, pyth_oracle=HovQMDrbAgAYPCmHVSrezcSmkMtXSSUsLDFANExrZh2J),
 Market(symbol='ETH-PERP', base_asset_symbol='ETH', market_index=2, pyth_oracle=EdVCmQ9FSPcVe5YySXDPCRmc8aDQLKJ9xvYBMZPie1Vw)]

In [3]:
# def orderbook_all(perp_market="SOL-PERP"):
#     """ Queries DEVNET orderbok for specified perp market, organizes data into bid/asks sorted
#     """
#     try:
#         r = requests.get(URL_DEVNET_OB).json()
#     except:
#         print(r)
#         raise Exception("Unable to fetch orderbook")

#     slot    = r['slot']
#     oracles = r['oracles']
#     orders  = r['orders']

#     ## market index -> price
#     market_to_oracle_map = pd.DataFrame(r['oracles']).set_index('marketIndex').to_dict()['price']

#     ## Get list of outstanding orders    
#     orders_all = list(map(lambda d: {"user":d['user'][:10], **d.pop('order')}, orders))
#     df = pd.DataFrame(orders_all)
#     df_raw = pd.DataFrame(orders_all)
    
#     ## Only interested in (a) SOL-PERP (b) limit orders
#     # SOL is marketINdex=0
#     df = df[(df.marketIndex == MARKET_INDEX_SOL) & (df.marketType=='perp')]
    
#     df["oraclePrice"] = df["marketIndex"].apply(lambda x: market_to_oracle_map.get(x, None))

#     ## Convert from lamports:
#     for col in ['price', 'oraclePrice', 'oraclePriceOffset']:
#         df[col] = df[col].astype(int)
#         df[col] /= 1e6

#     for col in ['quoteAssetAmountFilled']:
#         df[col] = df[col].astype(int)
#         df[col] /= 1e6 

#     for col in ['baseAssetAmount', 'baseAssetAmountFilled']:
#         df[col] = df[col].astype(int)
#         df[col] /= 1e9
        
    
#     bid_df = df[(df.direction == 'long') & (df.orderType == 'limit')]
#     ask_df = df[(df.direction == 'short')   &( df.orderType == 'limit')]
    
#     ## Fixed prices are ok
#     ## Floating prices need to be marked to a fixed price according to oracle price    
#     bid_float = bid_df.loc[bid_df.price == 0] # inv: 0 price <-> floating order .w.r.t oracle 
#     ask_float = ask_df.loc[ask_df.price == 0] #
    
#     bid_df.loc[bid_df.price == 0, "price"] = bid_float["oraclePrice"] + bid_float["oraclePriceOffset"]
#     ask_df.loc[ask_df.price == 0, "price"] = ask_float["oraclePrice"] + ask_float["oraclePriceOffset"]
    
    
#     bid_df = bid_df.sort_values(['price'], ascending=False)
#     ask_df = ask_df.sort_values(['price'])
    
#     bid_df = bid_df.reset_index(drop=True)
#     ask_df = ask_df.reset_index(drop=True)
    
#     return df_raw, bid_df, ask_df

# def orderbook_expanded(perp_market="SOL-PERP"):
#     """
#         simplieid view of orderbook
#     """
#     _, bid, ask = orderbook_all(perp_market)
    
    
    
#     bid_simple = bid[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]
#     ask_simple = ask[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]

#     #### SOME ARE PARTIALLY FILLED
#     bid_simple["qty"] = bid_simple["baseAssetAmount"] - bid_simple["baseAssetAmountFilled"]
#     ask_simple["qty"] = ask_simple["baseAssetAmount"] - ask_simple["baseAssetAmountFilled"]
    

#     return bid_simple, ask_simple

# def get_dlob(perp_market="SOL-PERP"):
#     """
#         Returns simplified orderbook view 
#     """
#     bid, ask = orderbook_expanded(perp_market)
    
#     bid_ob = bid[["price", "qty"]].groupby("price", sort=False).sum()
#     ask_ob = ask[["price", "qty"]].groupby("price", sort=False).sum()
#     return bid_ob, ask_ob
    
# def fair_price_dlob(bid_ob, ask_ob):
#     return (bid_ob.index.max() + ask_ob.index.min() ) / 2.0
    

# df_raw, bid, ask = orderbook_all()
# print(f"#bids={len(bid)}, #asks={len(ask)}")
# bid_s, ask_s     = orderbook_expanded()
# bid_ob, ask_ob   = get_dlob()

In [4]:
# async def get_solana_perp_oracle():
#     """ return SOL-PERP oracle """
#     with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
#     kp = Keypair.from_secret_key(bytes(secret))
#     subaccount = 0
#     print('using public key:', kp.public_key, " subaccount = ", subaccount)

#     env = 'devnet'
#     config = configs[env]
#     wallet = Wallet(kp) 
#     connection = AsyncClient(config.default_http)
#     provider = Provider(connection, wallet)

#     ch = ClearingHouse.from_config(config, provider)
#     chu = ClearingHouseUser(ch, authority=ch, subaccount_id=subaccount)
#     perp_market = await chu.get_perp_market(0)
#     oracle = (await chu.get_perp_oracle_data(perp_market)).price / PRICE_PRECISION

#     return oracle

In [5]:
def mm_order_single(order_offset, order_direction=PositionDirection.LONG(), order_qty=0):

    return OrderParams(
        order_type=OrderType.LIMIT(),
        market_type=MarketType.PERP(),
        direction=order_direction,
        user_order_id=0,
        base_asset_amount = int(order_qty * BASE_PRECISION),
        price=0,
        market_index=0,
        reduce_only=False,
        post_only=PostOnlyParams.TRY_POST_ONLY(),
        immediate_or_cancel=False,
        trigger_price=0,
        trigger_condition=OrderTriggerCondition.ABOVE(),
        oracle_price_offset= int(order_offset * PRICE_PRECISION),
        auction_duration=None,
        max_ts=None,
        auction_start_price=None,
        auction_end_price=None,
    )


async def construct_and_place(bid_offsets, bid_qtys, ask_offsets, ask_qtys, cancel_existing=True):
    """
        Bundles offsets / quantities for bids / asks into an array of orders
        and places order
        
        cancels existing orders before placing
    """
    all_orders = []
    
    if(cancel_existing):
        all_orders.append(await drift_acct.get_cancel_orders_ix(0))
    
    for bo, bq in zip(bid_offsets, bid_qtys):
        if(bq > 0):
            temp_order = mm_order_single(bo, PositionDirection.LONG(), bq)
            all_orders.append(
                await drift_acct.get_place_perp_order_ix(temp_order, 0)
            )
    for ao, aq in zip(ask_offsets, ask_qtys):
        if(aq > 0):
             all_orders.append(
                 await drift_acct.get_place_perp_order_ix(mm_order_single(ao, PositionDirection.SHORT(), aq), 0)
             )
        
    ret = await drift_acct.send_ixs(all_orders)
    
    return ret


bid_offsets = [-10, -12, -15]
bid_qtys    = [1.1, 2.2, 1.1]

ask_offsets = [10, 12, 15]
ask_qtys    = [1.1, 2.2, 1.1]

# await construct_and_place(bid_offsets, bid_qtys, ask_offsets, ask_qtys)

# Market Making Parameters

In [13]:
## Set up clearing house user client to view balances 

subaccount_id = 0
env = 'devnet'
url = 'https://api.devnet.solana.com'
with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
kp = Keypair.from_secret_key(bytes(secret))
config = configs[env]
wallet = Wallet(kp)
connection = AsyncClient(url)
provider = Provider(connection, wallet)
drift_acct = ClearingHouse.from_config(config, provider)
chu        = ClearingHouseUser(drift_acct, use_cache=True)

/home/sjl/Projects/drift/driftpy-examples/venv/lib/python3.10/site-packages/driftpy/idl/drift.json


In [14]:
# Constants that never change 

## Initial deposit into account
INITIAL_COLLATERAL_SOL = 1
INITIAL_COLLATERAL_USD = 100

TARGET_MAX_SIZE = 100
LEVERAGE_LIMIT  = 2

# Computed from above:
curr_collateral = None
curr_leverage   = None

AGGRESSION = 50e-4

In [19]:
async def get_stats(chu):
    perp_market = await chu.get_perp_market(0)
    return {
        "unrealized_pnl" : await chu.get_unrealized_pnl() / PRICE_PRECISION,
        "oracle_price"   : (await chu.get_perp_oracle_data(perp_market)).price / PRICE_PRECISION,
        "curr_leverage"  : ( await chu.get_leverage() ) / 10_000,  # h/t bigz,
        "SOL_PERP_pos"   : (await chu.get_user_position(0))
    }

In [24]:
history = {}

In [25]:
# tob = bid_ob.iloc[:3].reset_index()
# toa = ask_ob.iloc[:3].reset_index()

# bid_avg = (tob["price"] * tob["qty"]).sum() / tob["qty"].sum()
# ask_avg = (toa["price"] * toa["qty"]).sum() / toa["qty"].sum()

# print(bid_avg, ask_avg)

In [31]:
from simple_mm import *

epoch = 0
while epoch < 3:
    
    print("Epoch: ", epoch)
    epoch += 1
    
    ## ---------- Pull stats  --------------
    await chu.set_cache()
    stats = await get_stats(chu)
    print(stats)
    
    # (1) Current SOL-PERP position
    pp = stats['SOL_PERP_pos'] #await chu.get_user_position(0)
    current_pos = pp.base_asset_amount / 1e9  
    print(f"> current pos = {current_pos}")
    
    # (2): Get the current order book (GET Request)
    bid_ob, ask_ob = get_dlob()

    # (3): Calculate the fair price based on the order book
    metrics = dlob_metrics(bid_ob, ask_ob, stats['oracle_price'])
    print(f"> dlob metrics = {metrics}")
    
    ## PART 2: Prepare new bids / ask price points
    
    #### Signals --- very simple ----
    
    ## check if overleveraged -- here we need to reduce position immediately
    over_leveraged = True if stats['curr_leverage'] > 0.8 * LEVERAGE_LIMIT else False
    
    ## Backoff position if needed
    engage_sell_pressure  = True if current_pos > TARGET_MAX_SIZE  * 0.8 else False # Hyperparemeter 
    engage_buy_pressure   = True if current_pos < -TARGET_MAX_SIZE * 0.8 else False # Hyperparemeter 
    
    ## no bid / no ask
    no_bid = True if over_leveraged and engage_sell_pressure else False
    no_ask = True if over_leveraged and engage_buy_pressure else False
    
        
    ## Adjustment on buy / sell spread due to buying and selling pressure in orderbook
    if(metrics['bid_avg'] <= stats['oracle_price'] <= metrics['ask_avg']):
        
        ask_to_bid_factor = (metrics['ask_avg'] - stats['oracle_price']) / (stats['oracle_price'] - metrics['bid_avg'])
        if(ask_to_bid_factor > 1):
            ask_adj_factor = 1 + np.ln(ask_to_bid_factor)
        elif(ask_to_bid_factor < 1):
            bid_adj_factor = 1 + np.ln(ask_to_bid_factor)
    
    # ------ bid side -------
    ## Compute total size
    bid_target_pos     = max(0, TARGET_MAX_SIZE - current_pos)
    bid_offset_width   = AGGRESSION 
    if(engage_buy_pressure):
        bid_offset_width *= 0.5 #Hyperparemeter
        
    bid_offsets        = np.linspace(0, bid_offset_width, 4)[1:]
    bid_qts            = bid_target_pos * np.array([0.25, 0.5, 0.25])
    
    print(f"> bid_target_pos = {bid_target_pos}")
    print(f"> bid_offsets = {bid_offsets}")
    print(f"> bid_qts = {bid_qts}")
    
    # ------ ask side -------
    ask_target_pos   = max(0, TARGET_MAX_SIZE + current_pos)
    ask_offset_width = AGGRESSION
    if(engage_sell_pressure):
        ask_offset_width *= 0.5
    
    ask_offsets      = np.linspace(0, ask_offset_width, 4)[1:]
    ask_qts          = ask_target_pos * np.array([0.25, 0.5, 0.25])
    print(f"> ask_target_pos = {ask_target_pos}")
    print(f"> ask_prices = {ask_offsets}")
    print(f"> ask_qts = {ask_qts}")
    
    ## ---------- Send order  --------------
    ret = await construct_and_place(bid_offsets, bid_qts, ask_offsets, ask_qts)
    print(ret)
    
    ## ---------- Logging --------------    
    # Step 6: Wait for 1 second before repeating the loop
    await asyncio.sleep(1)

Epoch:  0
{'unrealized_pnl': -53.460049, 'oracle_price': 23.19025, 'curr_leverage': 0.15780714633391832, 'SOL_PERP_pos': PerpPosition(last_cumulative_funding_rate=-3291315921, base_asset_amount=-285440000000, quote_asset_amount=6565964911, quote_break_even_amount=6569718709, quote_entry_amount=6568139351, open_bids=385420000000, open_asks=0, settled_pnl=0, lp_shares=0, last_base_asset_amount_per_lp=0, last_quote_asset_amount_per_lp=0, remainder_base_asset_amount=0, market_index=0, open_orders=3, padding=ListContainer([0]))}
> current pos = -285.44
> dlob metrics = {'fair_price': 3.406125000000003, 'bid': 0.002500000000001279, 'ask': 6.809750000000001, 'bid_avg': 0.0016573591406761068, 'ask_avg': 6.809750000000001}
> bid_target_pos = 385.44
> bid_offsets = [0.00083333 0.00166667 0.0025    ]
> bid_qts = [ 96.36 192.72  96.36]
> ask_target_pos = 0
> ask_prices = [0.00166667 0.00333333 0.005     ]
> ask_qts = [0. 0. 0.]
5gCGiHiJhr9BTrZAiSJxteQhYVRMzbhooNTY9Ntu2TKeGYg91XE8akZLyTM1k3MfKRT7TS